In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from PIL import Image

import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.torch_core import *
from fastai.callback import *
from fastai.basic_train import *
import torchvision.models as tmodels

from PIL import Image


import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import ToTensor,Resize

from PIL import Image
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import time
import sys
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func

from sklearn.metrics.ranking import roc_auc_score
import sklearn.metrics as metrics
import random



use_gpu = torch.cuda.is_available()

In [2]:
baseFolder  = "/home/santhosr/Documents/Chexpert"

cols = ['Path',
 'Sex',
 'Age',
 'View',
 'AP/PA',
 'No_Finding',
 'Enlarged_Cardiomediastinum',
 'Cardiomegaly',
 'Lung_Opacity',
 'Lung_Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Effusion',
 'Pleural_Other',
 'Fracture',
 'Support_Devices']

### Utils


In [3]:
def cleanLabel(x):
    
    labelCount = 0
    
    if x.Pleural_Effusion == 1:
        labelCount += 1
        
    if x.Edema == 1:
        labelCount += 1
    
    if x.Cardiomegaly ==1:
        labelCount += 1
    
    if x.Pneumonia == 1:
        labelCount += 1
    
    return labelCount
    
    

def getLabel(x,disease):
    
    if x[disease] ==1:
        return disease
    else:
        return "Rest"


def cleanLabelFile(df, disCols):
    
    for i in range(len(disCols)):

        #Changing NaN labels
        df.loc[pd.isna(df[disCols[i]]) , disCols[i]] = 0.0

        #Changing unsure labels
        df.loc[ df[disCols[i]]==-1.0 , disCols[i]] = 1.0

    return df


def getLabelDf(x,df, labelMap=None, disease=None):
    
    x = x[36:]          #To account for the extra "././" added before the Path variable
    
    if disease == None:
        x = df.loc[df.Path == x] 
        return labelMap[x.label.values[0]]
    else:
        x = df.loc[df.Path == x] 
        if x.label.values[0]==disease:
            return 1
        else:
            return 0
        
        
def create1vAllData(df, disease):
    
    t = df[df[disease] == 1.0]
    f = df[df[disease] == 0.0]
    
    num_t = len(t)
    
    f = f.sample(n= num_t, replace = False)
    
    out = pd.concat([t,f])
    
    out = out.sample(frac = 1.0, replace = False)
    
    return out




def cleanLabelFile(df, disCols):
    
    for i in range(len(disCols)):

        #Changing NaN labels
        df.loc[pd.isna(df[disCols[i]]) , disCols[i]] = 0.0

        #Changing unsure labels
        df.loc[ df[disCols[i]]==-1.0 , disCols[i]] = 1.0

    return df

### Main


In [6]:
trainFile = pd.read_csv(os.path.join(baseFolder,'train.csv'), names = cols, header=0)
validFile = pd.read_csv(os.path.join(baseFolder,'valid.csv'), names = cols, header=0)

In [7]:
trainFile.head()

,Path,Sex,Age,View,AP/PA,No_Finding,Enlarged_Cardiomediastinum,Cardiomegaly,Lung_Opacity,Lung_Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural_Effusion,Pleural_Other,Fracture,Support_Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [8]:
trainFile["Path"] = trainFile.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])
validFile["Path"] = validFile.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])


In [9]:
selectCols = ['Path',"View",'Sex',"Pleural_Effusion", "Edema","Cardiomegaly","Pneumonia"]

trainFile = trainFile[selectCols]
validFile = validFile[selectCols]

In [10]:
# -1 for Uncertain, 0 for negative, 1 for positive

trainFile['isClean'] = trainFile.apply(lambda x : cleanLabel(x), axis = 1)
validFile['isClean'] = validFile.apply(lambda x : cleanLabel(x), axis = 1)

trainFile['train'] = False
validFile['train'] = True

trainFile = trainFile[trainFile.isClean==1]
validFile = validFile[validFile.isClean==1]

In [11]:
df = pd.concat([trainFile,validFile])

df['label'] = df.apply(lambda x : getLabel(x), axis = 1)

In [12]:
df.label.value_counts()

Pleural_Effusion    52629
Edema               19405
Cardiomegaly         9259
Pneumonia            3383
Name: label, dtype: int64

In [13]:
labelMap = {"Pleural_Effusion":0, "Edema":1,"Cardiomegaly":2,"Pneumonia":3}

def getLabelDf(x):
    
#     print(x)
    
    x = x[36:]          #To account for the extra "././" added before the Path variable
#     print(x)
    x = df.loc[df.Path == x] 
#     print(x)
    
    return labelMap[x.label.values[0]]

In [18]:
df.iloc[0].Path

'train/patient00003/study1/view1_frontal.jpg'

In [14]:
df.Path[0]

'valid/patient64541/study1/view1_frontal.jpg'

In [21]:
a = '/home/santhosr/Documents/Chexpert/.'
len(a)

35

In [32]:
df[df.Path=='train/patient00003/study1/view1_frontal.jpg']

,Path,View,Sex,Pleural_Effusion,Edema,Cardiomegaly,Pneumonia,isClean,train,label
4,train/patient00003/study1/view1_frontal.jpg,Frontal,Male,NaN,1.0,NaN,NaN,1,False,Edema


### Modeling

In [27]:
baseFolder  = baseFolder+"/"

In [ ]:
data = ImageItemList.from_df(df=df,path=baseFolder, cols='Path').no_split().label_from_func(getLabelDf).transform(get_transforms(),size=256).databunch(bs=100).normalize()

ImageDataBunch;

Train: LabelList
y: CategoryList (1000 items)
[Category Edema, Category Cardiomegaly, Category Pleural_Effusion, Category Pleural_Effusion, Category Cardiomegaly]...
Path: /home/santhosr/Documents/Chexpert
x: ImageItemList (1000 items)
[Image (3, 320, 390), Image (3, 320, 390), Image (3, 320, 379), Image (3, 320, 390), Image (3, 320, 369)]...
Path: /home/santhosr/Documents/Chexpert;

Valid: LabelList
y: CategoryList (0 items)
[]...
Path: /home/santhosr/Documents/Chexpert
x: ImageItemList (0 items)
[]...
Path: /home/santhosr/Documents/Chexpert;

Test: None

In [8]:
trainFileList = []

for i in range(len(trainPatientList)):
    
    f = os.listdir( os.path.join(baseFolder,'train',trainPatientList[i]) )
    
    if len(f)!=2:
        print(trainPatientList[i])
    
    trainFileList.append(f)
    
    
    


patient29677
patient39924
patient63268
patient14306
patient16501
patient46074
patient22556
patient63923
patient34155
patient21785
patient13174
patient07000
patient34662
patient53810
patient39870
patient02890
patient41368
patient01586
patient36033
patient22813
patient23560
patient28595
patient53915
patient37981
patient18055
patient17438
patient05265
patient51266
patient24489
patient17552
patient52926
patient29252
patient40185
patient49198
patient04418
patient03337
patient56009
patient00838
patient60623
patient03753
patient05978
patient10205
patient38533
patient02706
patient06355
patient26647
patient61955
patient33468
patient07456
patient35469
patient58797
patient12191
patient57011
patient50758
patient30078
patient59816
patient23103
patient59906
patient26384
patient61027
patient01427
patient49585
patient25978
patient06002
patient54698
patient34362
patient54862
patient41099
patient41928
patient35786
patient49802
patient59078
patient23263
patient59948
patient25310
patient02569
patient30020

patient52474
patient13759
patient20970
patient19451
patient55963
patient43931
patient63490
patient25022
patient01945
patient30839
patient52034
patient60709
patient03317
patient43794
patient18853
patient27235
patient05317
patient32560
patient03404
patient00291
patient41291
patient19423
patient46165
patient10413
patient25601
patient35901
patient59290
patient45433
patient09861
patient46521
patient62694
patient13975
patient09259
patient06318
patient64106
patient47285
patient47463
patient11624
patient10097
patient32936
patient23227
patient36797
patient57086
patient04627
patient33493
patient22738
patient60990
patient62882
patient14611
patient49237
patient60989
patient49917
patient46118
patient32952
patient50458
patient46354
patient50685
patient04027
patient50596
patient03629
patient45936
patient41989
patient57493
patient38839
patient25958
patient08800
patient31017
patient46483
patient58249
patient09161
patient54661
patient35766
patient32153
patient46994
patient23583
patient14426
patient35360

patient24826
patient18437
patient40544
patient00200
patient57465
patient60104
patient32142
patient10085
patient25773
patient51209
patient11913
patient09545
patient49991
patient11449
patient08573
patient26165
patient57576
patient08083
patient62243
patient48754
patient13232
patient60758
patient01060
patient19293
patient16154
patient14829
patient27069
patient35801
patient60194
patient39398
patient32601
patient40965
patient60985
patient59834
patient56937
patient19037
patient00964
patient42017
patient45045
patient07633
patient59458
patient14510
patient08626
patient37769
patient44925
patient55987
patient47739
patient56405
patient35405
patient56797
patient30314
patient40006
patient54918
patient39590
patient27424
patient58673
patient51735
patient48811
patient05074
patient28546
patient00464
patient25110
patient38679
patient22353
patient39061
patient11208
patient62930
patient29048
patient15810
patient44232
patient37073
patient16394
patient49317
patient43092
patient09144
patient19381
patient26944

patient01663
patient34668
patient33982
patient04425
patient02029
patient26640
patient34301
patient25596
patient23526
patient33251
patient53754
patient50354
patient02811
patient07414
patient49412
patient00243
patient48125
patient00976
patient54873
patient27625
patient28571
patient56369
patient35182
patient04790
patient42325
patient10279
patient03433
patient09338
patient58021
patient63599
patient63337
patient58113
patient20292
patient42416
patient41116
patient29824
patient41295
patient32532
patient16153
patient50411
patient27102
patient30886
patient34430
patient20710
patient36313
patient03148
patient02976
patient11616
patient46975
patient22347
patient52101
patient19337
patient49976
patient15918
patient61237
patient13532
patient29809
patient60291
patient60644
patient52543
patient08319
patient51699
patient07744
patient34024
patient61050
patient01939
patient63564
patient58992
patient15914
patient43863
patient49884
patient22695
patient22759
patient03389
patient46558
patient52408
patient10023

patient32931
patient14996
patient20470
patient25767
patient30073
patient10440
patient20003
patient31707
patient04637
patient62696
patient28462
patient17527
patient42634
patient48961
patient63591
patient48256
patient19645
patient14905
patient30006
patient12428
patient25759
patient43163
patient40105
patient44521
patient09610
patient43668
patient20572
patient10562
patient36250
patient37405
patient42965
patient23277
patient40376
patient07734
patient26077
patient05200
patient35456
patient61350
patient22139
patient46774
patient05005
patient01666
patient58007
patient15398
patient22284
patient38833
patient43348
patient58116
patient09203
patient28018
patient63251
patient13323
patient55230
patient50732
patient38600
patient22291
patient32385
patient13864
patient30866
patient13062
patient32156
patient13506
patient44222
patient15647
patient42704
patient64013
patient51324
patient61581
patient63885
patient06891
patient17600
patient28115
patient20643
patient49035
patient01715
patient02813
patient34887

...

patient50449
patient36262
patient49529
patient18687
patient33200
patient17773
patient07550
patient52284
patient16941
patient05717
patient15056
patient59472
patient34217
patient35389
patient15899
patient24890
patient23128
patient38476
patient08366
patient62471
patient29453
patient38655
patient03098
patient63420
patient62921
patient00223
patient52273
patient54361
patient29592
patient35191
patient58890
patient55132
patient25169
patient01858
patient39356
patient19995
patient47925
patient24395
patient13336
patient57124
patient22687
patient53727
patient09110
patient28587
patient58318
patient49165
patient13001
patient62269
patient56993
patient52845
patient05981
patient41979
patient46185
patient05846
patient36518
patient57695
patient03860
patient51073
patient29115
patient35680
patient00129
patient49511
patient62165
patient00310
patient14397
patient58147
patient00550
patient45069
patient33212
patient51613
patient60599
patient01176
patient28922
patient60670
patient06956
patient13845
patient55594

patient61133
patient28441
patient36369
patient16059
patient49017
patient44887
patient39761
patient35392
patient36524
patient30071
patient48272
patient15039
patient55593
patient33461
patient55864
patient51008
patient11360
patient28988
patient49460
patient34837
patient31875
patient07187
patient63320
patient44923
patient06435
patient63861
patient56593
patient61768
patient09497
patient04430
patient45921
patient30519
patient37368
patient61707
patient17483
patient21728
patient18042
patient60918
patient43223
patient13280
patient52745
patient29355
patient15413
patient04813
patient48615
patient37479
patient28758
patient23054
patient08567
patient10947
patient38547
patient60499
patient08143
patient63269
patient53836
patient22262
patient02135
patient27225
patient20866
patient59553
patient53546
patient14227
patient23760
patient30416
patient29113
patient21108
patient56280
patient06274
patient38307
patient21372
patient52094
patient12370
patient03121
patient60541
patient00050
patient46236
patient26314

patient53319
patient38954
patient30262
patient36312
patient45700
patient47719
patient44176
patient46390
patient05281
patient58850
patient21292
patient27376
patient07487
patient49981
patient25532
patient31135
patient20485
patient02956
patient21018
patient17129
patient27577
patient08491
patient02142
patient59005
patient63894
patient43607
patient13444
patient21678
patient36782
patient27185
patient40080
patient23735
patient44275
patient07110
patient07525
patient61436
patient58602
patient37890
patient29664
patient10218
patient31748
patient62897
patient36895
patient20599
patient08294
patient47293
patient30553
patient35410
patient20939
patient02523
patient10854
patient10713
patient15736
patient46067
patient27051
patient45261
patient45547
patient32186
patient40600
patient25500
patient04402
patient43414
patient62161
patient64264
patient11210
patient41070
patient11614
patient04404
patient16652
patient25553
patient15138
patient47141
patient58407
patient17478
patient02230
patient13502
patient41237

patient54508
patient55462
patient32071
patient58413
patient26181
patient48868
patient06439
patient19022
patient18536
patient04909
patient13173
patient52025
patient59461
patient37255
patient20639
patient55631
patient47454
patient58998
patient28870
patient23422
patient47321
patient32968
patient23853
patient25493
patient05197
patient34929
patient37681
patient57562
patient54593
patient49224
patient04393
patient23692
patient28312
patient42538
patient23719
patient13448
patient29785
patient12871
patient47959
patient25613
patient27641
patient27934
patient54028
patient11990
patient56449
patient49673
patient64334
patient54097
patient60675
patient31388
patient00450
patient07641
patient54764
patient56359
patient45920
patient03049
patient42645
patient57073
patient15435
patient63653
patient36135
patient06155
patient43880
patient17067
patient28351
patient34645
patient46286
patient60280
patient60224
patient16981
patient38046
patient04658
patient33306
patient52668
patient47079
patient17763
patient33231

patient58371
patient30467
patient38324
patient42476
patient43482
patient55992
patient10377
patient22475
patient37119
patient22091
patient56422
patient02369
patient01734
patient33581
patient38950
patient42777
patient62685
patient53679
patient34861
patient26704
patient36142
patient63657
patient53191
patient54636
patient19435
patient05220
patient54033
patient36007
patient35228
patient03125
patient43200
patient43745
patient48377
patient45299
patient36044
patient54960
patient49323
patient37648
patient07282
patient21278
patient05019
patient11584
patient45677
patient34222
patient00784
patient48103
patient47405
patient53947
patient60409
patient49178
patient47603
patient08318
patient15218
patient02742
patient16210
patient25622
patient55587
patient29706
patient20083
patient05253
patient37891
patient53559
patient53366
patient29023
patient62948
patient40611
patient25193
patient47051
patient06873
patient18796
patient57857
patient07516
patient27997
patient36390
patient15528
patient17250
patient21817

In [10]:
lenList = []

for i in range(len(trainFileList)):
    
    lenList.append(len(trainFileList[i]))

In [13]:
df = pd.DataFrame({'patient':trainPatientList, "numFiles":lenList})

In [14]:
df.head()

,patient,numFiles
0,patient29677,1
1,patient54752,2
2,patient39924,4
3,patient63268,3
4,patient23509,2


In [15]:
df[df.numFiles==55]

,patient,numFiles
64053,patient32500,55


In [17]:
os.listdir(os.path.join(baseFolder,'train','patient32500'))

['study49',
 'study40',
 'study3',
 'study22',
 'study47',
 'study20',
 'study38',
 'study50',
 'study31',
 'study16',
 'study36',
 'study32',
 'study30',
 'study25',
 'study37',
 'study19',
 'study42',
 'study23',
 'study28',
 'study9',
 'study17',
 'study5',
 'study44',
 'study1',
 'study10',
 'study21',
 'study35',
 'study29',
 'study55',
 'study7',
 'study18',
 'study45',
 'study4',
 'study26',
 'study27',
 'study53',
 'study11',
 'study46',
 'study14',
 'study8',
 'study15',
 'study12',
 'study39',
 'study33',
 'study24',
 'study41',
 'study52',
 'study6',
 'study13',
 'study2',
 'study43',
 'study48',
 'study34',
 'study54',
 'study51']

In [77]:
a = Image.open('/home/santhosr/Documents/Chexpert/./train/patient00003/study1/view1_frontal.jpg')

In [78]:
e=a.convert('RGB')

In [80]:
r=np.array(e)

In [81]:
r.shape

(320, 390, 3)

In [20]:
trainFile

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
9,CheXpert-v1.0-small/train/patient00005/study2/...,Male,33,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [22]:
trainFile.Path[0]

'CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg'

In [23]:
trainFile.Path[1]

'CheXpert-v1.0-small/train/patient00002/study2/view1_frontal.jpg'

In [24]:
trainFile.Path[2]

'CheXpert-v1.0-small/train/patient00002/study1/view1_frontal.jpg'

### Dataloader

In [44]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import ToTensor,Resize

from PIL import Image
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [82]:
class LungDataset(Dataset):
    

    def __init__(self, csvFile, rootDir, transform = None):
        """
        Args:
            rootDir : Directory that has train, valid, train.csv and valid.csv
            csvFile : train.csv or valid.csv
        """
        
        self.rootDir = rootDir
        self.transform = transform
        
        cols = ['Path',
                 'Sex',
                 'Age',
                 'View',
                 'AP/PA',
                 'No_Finding',
                 'Enlarged_Cardiomediastinum',
                 'Cardiomegaly',
                 'Lung_Opacity',
                 'Lung_Lesion',
                 'Edema',
                 'Consolidation',
                 'Pneumonia',
                 'Atelectasis',
                 'Pneumothorax',
                 'Pleural_Effusion',
                 'Pleural_Other',
                 'Fracture',
                 'Support_Devices']
        
        self.df = pd.read_csv(os.path.join(rootDir,csvFile), names = cols, header=0)
        
        
        #Modifying the path variable
        self.df["Path"] = self.df.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])
        
        #retaining important vars
        selectCols = ['Path',"View",'Sex',"Pleural_Effusion", "Edema","Cardiomegaly","Pneumonia"]
        self.df = self.df[selectCols]
        
        self.df['isClean'] = self.df.apply(lambda x : cleanLabel(x), axis = 1)

        #Retaining only samples with 1 disease
        self.df = self.df[self.df.isClean==1]
        
        #Creating the label variable
        self.df['label'] = self.df.apply(lambda x : labelMap[getLabel(x)], axis = 1)
                



    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        imgPath = os.path.join( self.rootDir, self.df.iloc[idx].Path)
        image = Image.open(imgPath).convert('RGB')
        
        label = self.df.iloc[idx].label

        if self.transform:
            image = self.transform(image)

        return image, label

In [83]:
trainDataset = LungDataset('train.csv', baseFolder, transforms.Compose([Resize((256,256)), ToTensor()]))

In [62]:
e = next(iter(trainDataset))

In [63]:
e[0].size()

torch.Size([1, 256, 256])

In [86]:
trainLoader = DataLoader(trainDataset, batch_size= 12, shuffle =True, num_workers = 4)

### Resnet Transfer Learning

In [67]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

In [91]:
device = torch.device('cuda:0')

model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
num_ftrs = 2048
model_ft.fc = nn.Linear(num_ftrs, 4)

model_ft.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [92]:
maxEpochs = 5

optimizer = optim.SGD(model_ft.parameters(), lr = 0.001)

for epoch in range(maxEpochs):
    
    for image, label in trainLoader:
        
        image, label = image.to(device), label.to(device)
#         print(image.size())
        optimizer.zero_grad()
        
        output = model_ft(image)
    
        celoss = nn.CrossEntropyLoss()

        loss = celoss(output, label)

        loss.backward()

        optimizer.step()

    
        
        

torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/santhosr/.conda/envs/fastai/lib/

KeyboardInterrupt: 

In [89]:
3*256*256

196608

In [72]:
help(Image.open)

Help on function open in module PIL.Image:

open(fp, mode='r')
    Opens and identifies the given image file.
    
    This is a lazy operation; this function identifies the file, but
    the file remains open and the actual image data is not read from
    the file until you try to process the data (or call the
    :py:meth:`~PIL.Image.Image.load` method).  See
    :py:func:`~PIL.Image.new`. See :ref:`file-handling`.
    
    :param fp: A filename (string), pathlib.Path object or a file object.
       The file object must implement :py:meth:`~file.read`,
       :py:meth:`~file.seek`, and :py:meth:`~file.tell` methods,
       and be opened in binary mode.
    :param mode: The mode.  If given, this argument must be "r".
    :returns: An :py:class:`~PIL.Image.Image` object.
    :exception IOError: If the file cannot be found, or the image cannot be
       opened and identified.



## Multi-Label Classification

In [3]:
baseFolder  = "/home/santhosr/Documents/Chexpert"

cols = ['Path',
 'Sex',
 'Age',
 'View',
 'AP/PA',
 'No_Finding',
 'Enlarged_Cardiomediastinum',
 'Cardiomegaly',
 'Lung_Opacity',
 'Lung_Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Effusion',
 'Pleural_Other',
 'Fracture',
 'Support_Devices']

In [30]:
trainFile = pd.read_csv(os.path.join(baseFolder,'train.csv'), names = cols, header=0)
validFile = pd.read_csv(os.path.join(baseFolder,'valid.csv'), names = cols, header=0)

trainFile["Path"] = trainFile.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])
validFile["Path"] = validFile.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])


In [6]:
trainFile.head()

,Path,Sex,Age,View,AP/PA,No_Finding,Enlarged_Cardiomediastinum,Cardiomegaly,Lung_Opacity,Lung_Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural_Effusion,Pleural_Other,Fracture,Support_Devices
0,train/patient00001/study1/view1_frontal.jpg,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,train/patient00002/study2/view1_frontal.jpg,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,train/patient00002/study1/view1_frontal.jpg,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,train/patient00002/study1/view2_lateral.jpg,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,train/patient00003/study1/view1_frontal.jpg,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [31]:
def getTag(x):
    
    tag=''
    if x['Atelectasis'] == 1.0:
        tag += 'Atelectasis'
        tag +=","
        
    if x['Cardiomegaly'] == 1.0:
        tag += 'Cardiomegaly'
        tag +=","
    
    if x['Consolidation'] == 1.0:
        tag += 'Consolidation'
        tag +=","
    
    if x['Edema'] == 1.0:
        tag += 'Edema'
        tag +=","
        
    if x['Pleural_Effusion'] == 1.0:
        tag += 'PleuralEffusion'
        tag +=","
        
    
    tag = tag[:-1]
    
    return(tag)
    

In [32]:
trainFile['tag'] = trainFile.apply(lambda x : getTag(x), axis=1)
validFile['tag'] = validFile.apply(lambda x : getTag(x), axis=1)


In [33]:
trainFile['train'] = False
validFile['train'] = True

In [56]:
df = pd.concat([trainFile[:25000],validFile],axis =0)

In [57]:
df.columns

Index(['Path', 'Sex', 'Age', 'View', 'AP/PA', 'No_Finding',
       'Enlarged_Cardiomediastinum', 'Cardiomegaly', 'Lung_Opacity',
       'Lung_Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural_Effusion', 'Pleural_Other', 'Fracture',
       'Support_Devices', 'tag', 'train'],
      dtype='object')

In [58]:
trainFile.columns

Index(['Path', 'Sex', 'Age', 'View', 'AP/PA', 'No_Finding',
       'Enlarged_Cardiomediastinum', 'Cardiomegaly', 'Lung_Opacity',
       'Lung_Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural_Effusion', 'Pleural_Other', 'Fracture',
       'Support_Devices', 'tag', 'train'],
      dtype='object')

In [59]:
validFile.columns

Index(['Path', 'Sex', 'Age', 'View', 'AP/PA', 'No_Finding',
       'Enlarged_Cardiomediastinum', 'Cardiomegaly', 'Lung_Opacity',
       'Lung_Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural_Effusion', 'Pleural_Other', 'Fracture',
       'Support_Devices', 'tag', 'train'],
      dtype='object')

In [85]:
data = (ImageItemList.from_df(path=baseFolder, df=df, cols='Path')
        .split_from_df(col='train')
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_df('tag')
        #How to label? -> use the second column of the csv file and split the tags by ' '
        .transform(get_transforms(), size=128)
        #Data augmentation? -> use tfms with a size of 128
        .databunch(bs=60).normalize())

In [44]:
len(data.valid_ds)

234

### Modeling

In [ ]:
class ModelTrackerCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model when monitored quantity is best."
    def __init__(self, learn:Learner, path:str='/home/santhosr/Documents/Courses/CIS700/Project/models',id:int=None,monitor:str='val_loss', mode:str='auto',modelName:str='resnet50'):
        super().__init__(learn, monitor=monitor, mode=mode)
        
        self.bestAcc = 0.0001
        
        valAccList = []
        valLossList = []
        trainLossList = []
        self.folderPath = path
        self.id = id
        self.modelName = modelName
        super().__post_init__()

    def on_epoch_end(self, epoch, **kwargs:Any)->None:
        "Compare the value monitored to its best score and maybe save the model."

        acc = float(self.learn.recorder.metrics[epoch-1][0])
        val_loss = self.learn.recorder.val_losses[epoch-1]

        if acc>self.bestAcc:
            self.bestAcc = acc
            if self.id==None:
                fileName = 'model_'+self.modelName+'_acc'+str(int(acc*1000))+"_loss"+str(int(val_loss*1000))
            else:
                fileName = 'model_'+self.modelName+'_id' + str(self.id) + '_acc' + str(int(acc*1000)) + "_loss" + str(int(val_loss*1000))
            fileName = os.path.join(self.folderPath, fileName)
            self.learn.save(fileName)

In [61]:
learn = create_cnn(data, tmodels.resnet50, metrics=accuracy,pretrained=True)

In [82]:
learn.freeze()
learn.fit(1,slice(1e-4))

epoch,train_loss,valid_loss,accuracy
1,2.267646,2.533363,0.414530


In [63]:
learn.fit(1,slice(1e-4))

832

In [83]:
learn.recorder.metrics

[[tensor(0.4145)]]

In [65]:
learn.recorder.val_losses

[2.7563572, 2.4910753]

In [80]:
def createPlots(model, path, modelID):
    
    ### TRAIN LOSS
    plt.plot(model.recorder.losses,'r-')
    plt.ylabel("Train Loss")
    plt.xlabel("Iterations")
    plt.savefig(os.path.join(path,"train_loss_"+str(modelID)+".png"))
    plt.clf()
    plt.close()
    
    ### VAL LOSS
    plt.plot(model.recorder.val_losses,'r-')
    plt.ylabel("Validation Loss")
    plt.xlabel("Epochs")
    plt.savefig(os.path.join(path,"valid_loss_"+str(modelID)+".png"))
    plt.clf()
    plt.close()
    
    ### VAL ACCURACY
    plt.plot(model.recorder.metrics,'r-')
    plt.ylabel("Validation Acc")
    plt.xlabel("Epochs")
    plt.savefig(os.path.join(path,"valid_acc_"+str(modelID)+".png"))
    plt.clf()
    plt.close()

## One vs All

In [4]:
baseFolder  = "/home/santhosr/Documents/Chexpert"

cols = ['Path',
 'Sex',
 'Age',
 'View',
 'AP/PA',
 'No_Finding',
 'Enlarged_Cardiomediastinum',
 'Cardiomegaly',
 'Lung_Opacity',
 'Lung_Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Effusion',
 'Pleural_Other',
 'Fracture',
 'Support_Devices']

In [5]:
disease = 'Cardiomegaly'
baseFolder  = "/home/santhosr/Documents/Chexpert"
modelID = 13
batchSize = 50

In [6]:
trainFile = pd.read_csv('trainNew.csv')
validFile = pd.read_csv('validNew.csv')


trainFile = cleanLabelFile(trainFile, cols[5:])
validFile = cleanLabelFile(validFile, cols[5:])

trainFile['train'] = False
validFile['train'] = True

# Balancing the classes
trainFile = create1vAllData(trainFile,disease)


df = pd.concat([trainFile[:1000],validFile[:1000]])

getLabel = partial(getLabel, disease = disease)
df['label'] = df.apply(lambda x : getLabel(x), axis = 1)

labelMap = {"Pleural_Effusion":0, "Edema":1,"Cardiomegaly":2,"Pneumonia":3,"Rest":4,"Fracture":5}


In [33]:
getLabelDf = partial(getLabelDf, df = df, disease = disease)

print("Data Creation Start")
data = ImageItemList.from_df(df=df,path=baseFolder, cols='Path').split_from_df(col='train').label_from_func(getLabelDf).transform(get_transforms(),size=256).databunch(bs=batchSize).normalize()
print("Data Creation Complete")

Data Creation Start
Data Creation Complete


## Create Data Split

In [3]:
trainFile = pd.read_csv(os.path.join(baseFolder,'train.csv'), names = cols, header=0)
validFile = pd.read_csv(os.path.join(baseFolder,'valid.csv'), names = cols, header=0)

trainFile["Path"] = trainFile.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])
validFile["Path"] = validFile.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])


In [4]:
trainFile.head()

,Path,Sex,Age,View,AP/PA,No_Finding,Enlarged_Cardiomediastinum,Cardiomegaly,Lung_Opacity,Lung_Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural_Effusion,Pleural_Other,Fracture,Support_Devices
0,train/patient00001/study1/view1_frontal.jpg,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,train/patient00002/study2/view1_frontal.jpg,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,train/patient00002/study1/view1_frontal.jpg,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,train/patient00002/study1/view2_lateral.jpg,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,train/patient00003/study1/view1_frontal.jpg,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [7]:
trainFile.describe()

,Age,No_Finding,Enlarged_Cardiomediastinum,Cardiomegaly,Lung_Opacity,Lung_Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural_Effusion,Pleural_Other,Fracture,Support_Devices
count,223414.000000,22381.0,44839.000000,46203.000000,117778.000000,11944.000000,85956.000000,70622.000000,27608.000000,68443.000000,78934.000000,133211.000000,6492.000000,12194.000000,123217.000000
mean,60.430653,1.0,-0.035795,0.409346,0.848911,0.644508,0.456769,-0.183498,-0.461134,-0.005304,0.206540,0.559706,0.134011,0.688699,0.932680
std,17.820925,0.0,0.718442,0.769323,0.472571,0.691607,0.741785,0.753980,0.828249,0.990244,0.493529,0.648859,0.966183,0.565435,0.283377
min,0.000000,1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,49.000000,1.0,-1.000000,0.000000,1.000000,1.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,1.000000
50%,62.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,74.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,90.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
disCols = cols[5:]

In [68]:
valData = []

for i in range(len(disCols)):
    d = trainFile[trainFile[disCols[i]] == 1.0]
    select = np.random.choice(list(range(len(d))),200, replace=False)
    
    valData.append(d.iloc[select])

In [69]:
val = pd.concat(valData)
val = val.drop_duplicates()
len(val)

In [71]:
trainIndex = list(set(list(trainFile.index)).difference(set(list(val.index))))

In [72]:
train = trainFile.loc[trainIndex]

In [75]:
train.head()

,Path,Sex,Age,View,AP/PA,No_Finding,Enlarged_Cardiomediastinum,Cardiomegaly,Lung_Opacity,Lung_Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural_Effusion,Pleural_Other,Fracture,Support_Devices
1,train/patient00002/study2/view1_frontal.jpg,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,train/patient00002/study1/view1_frontal.jpg,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,train/patient00002/study1/view2_lateral.jpg,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,train/patient00003/study1/view1_frontal.jpg,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
5,train/patient00004/study1/view1_frontal.jpg,Female,20,Frontal,PA,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [78]:
validFile['original'] = True
val['original'] = False

In [79]:
val = pd.concat([validFile,val])

In [80]:
len(val)

3023

In [85]:
val = val.sample(frac = 1.0, replace=False)

In [86]:
len(val)

3023

In [87]:
len(val.drop_duplicates())

3023

In [88]:
val.to_csv("validNew_withOrig.csv", index=False, index_label=False)

In [89]:
val.drop("original", inplace = True, axis = 1)

In [91]:
val.to_csv("validNew.csv", index=False, index_label=False)

In [92]:
train.to_csv("trainNew.csv", index=False, index_label=False)

## Next Section

In [99]:
trainFile = pd.read_csv('trainNew.csv')

In [112]:
def cleanLabelFile(df, disCols):
    
    for i in range(len(disCols)):

        #Changing NaN labels
        df.loc[pd.isna(df[disCols[i]]) , disCols[i]] = 0.0

        #Changing unsure labels
        df.loc[ df[disCols[i]]==-1.0 , disCols[i]] = 1.0

    return df
    

In [113]:
df2 = cleanLabelFile(trainFile, disCols)

In [115]:
df2.describe()

,Age,No_Finding,Enlarged_Cardiomediastinum,Cardiomegaly,Lung_Opacity,Lung_Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural_Effusion,Pleural_Other,Fracture,Support_Devices
count,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000,220625.000000
mean,60.423991,0.136780,0.103121,0.156592,0.497274,0.046885,0.291916,0.189688,0.110382,0.300029,0.100573,0.437462,0.026747,0.042461,0.523961
std,17.816866,0.343615,0.304117,0.363416,0.499994,0.211393,0.454645,0.392055,0.313366,0.458271,0.300764,0.496075,0.161343,0.201639,0.499427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,62.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,74.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,90.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [116]:
trainFile.Lung_Lesion.value_counts()

0.0    210281
1.0     10344
Name: Lung_Lesion, dtype: int64

## PyTorch code

In [4]:
class LungDataset(Dataset):
    def __init__(self, baseFolder, file, transform=None, type="All"):
        
        self.baseFolder = baseFolder
        
        self.df = pd.read_csv(file)
        self.df = cleanLabelFile(self.df, cols[5:])
        
        self.transform = transform

    def __getitem__(self, index):
        
        x = self.df.iloc[index]
        path = os.path.join(self.baseFolder, x['Path'])
        image = Image.open(path).convert('RGB')
        
        label = np.array(x[cols[5:]])
        
        
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.FloatTensor(list(label))

    def __len__(self):
        return len(self.df)

In [8]:
batchSize = 15
img_size = (256, 256)

#Transformations
transformList = []
transformList.append(transforms.Resize(img_size))
transformList.append(transforms.ColorJitter(brightness=.05, hue=.05, saturation=.05))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))      
transformSequence=transforms.Compose(transformList)

In [9]:
trainData = LungDataset(baseFolder, 'trainNew.csv',transformSequence)
validData = LungDataset(baseFolder, 'validNew.csv',transformSequence)

In [10]:
trainLoader = DataLoader(trainData, batch_size= batchSize, shuffle = True)


validLoader = DataLoader(validData, batch_size= batchSize, shuffle = True)

In [18]:
a = next(iter(trainLoader))

In [19]:
a[0].size()

torch.Size([15, 3, 256, 256])

#### Modeling

In [24]:
device = torch.device('cuda:0')


class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        
        self.densenet = torchvision.models.densenet121(pretrained=True)
        self.densenet.classifier = nn.Sequential(
            nn.Linear(4096, num_classes),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        x = self.densenet(x)
        return x


In [23]:
model

Net(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, tra

In [25]:
num_classes = 14
learning_rate = 0.0001

model = Net(num_classes).cuda()
criterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.Adam (model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)



In [32]:
def train(net, optimizer, criterion, train_loader, test_loader, epochs, model_name, plot):
    model = net.to(device)
    total_step = len(train_loader)
    
    
    overall_step = 0
    for epoch in range(epochs):
    
        for i, (images, labels) in enumerate(train_loader):

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            _, prediction = torch.max(outputs,1)

            overall_step+=1
            if (i+1) % 200 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
#               acc,d,e = getAcc2(model, validLoader)
#               print("Val Acc : {}".format(acc))
            if plot:
              info = { ('loss_' + model_name): loss.item()}

              
    

In [33]:
train(model, optimizer, criterion, trainLoader, validLoader, 3 , 'frontal', True)

Epoch [1/3], Step [200/14709], Loss: 0.4535
Epoch [1/3], Step [400/14709], Loss: 0.4055
Epoch [1/3], Step [600/14709], Loss: 0.4160
Epoch [1/3], Step [800/14709], Loss: 0.3839
Epoch [1/3], Step [1000/14709], Loss: 0.4015
Epoch [1/3], Step [1200/14709], Loss: 0.3800
Epoch [1/3], Step [1400/14709], Loss: 0.4027
Epoch [1/3], Step [1600/14709], Loss: 0.3908
Epoch [1/3], Step [1800/14709], Loss: 0.3938
Epoch [1/3], Step [2000/14709], Loss: 0.3151
Epoch [1/3], Step [2200/14709], Loss: 0.3612
Epoch [1/3], Step [2400/14709], Loss: 0.4175
Epoch [1/3], Step [2600/14709], Loss: 0.2975
Epoch [1/3], Step [2800/14709], Loss: 0.4143
Epoch [1/3], Step [3000/14709], Loss: 0.3541
Epoch [1/3], Step [3200/14709], Loss: 0.2796
Epoch [1/3], Step [3400/14709], Loss: 0.3707
Epoch [1/3], Step [3600/14709], Loss: 0.4137
Epoch [1/3], Step [3800/14709], Loss: 0.3812
Epoch [1/3], Step [4000/14709], Loss: 0.3705
Epoch [1/3], Step [4200/14709], Loss: 0.3961
Epoch [1/3], Step [4400/14709], Loss: 0.3289
Epoch [1/3], S

Epoch [3/3], Step [7400/14709], Loss: 0.2923
Epoch [3/3], Step [7600/14709], Loss: 0.2928
Epoch [3/3], Step [7800/14709], Loss: 0.3320
Epoch [3/3], Step [8000/14709], Loss: 0.3430
Epoch [3/3], Step [8200/14709], Loss: 0.3283
Epoch [3/3], Step [8400/14709], Loss: 0.3365
Epoch [3/3], Step [8600/14709], Loss: 0.3657
Epoch [3/3], Step [8800/14709], Loss: 0.3452
Epoch [3/3], Step [9000/14709], Loss: 0.3580
Epoch [3/3], Step [9200/14709], Loss: 0.4131
Epoch [3/3], Step [9400/14709], Loss: 0.3707
Epoch [3/3], Step [9600/14709], Loss: 0.3266
Epoch [3/3], Step [9800/14709], Loss: 0.2786
Epoch [3/3], Step [10000/14709], Loss: 0.3559
Epoch [3/3], Step [10200/14709], Loss: 0.3840
Epoch [3/3], Step [10400/14709], Loss: 0.3423
Epoch [3/3], Step [10600/14709], Loss: 0.3096
Epoch [3/3], Step [10800/14709], Loss: 0.3602
Epoch [3/3], Step [11000/14709], Loss: 0.2920
Epoch [3/3], Step [11200/14709], Loss: 0.3735
Epoch [3/3], Step [11400/14709], Loss: 0.4115
Epoch [3/3], Step [11600/14709], Loss: 0.4138
E

In [34]:
torch.save(model.state_dict(), "densenet_model1_3epochs.pt")

### Testing

In [ ]:
from sklearn.metrics import auc, roc_auc_score, accuracy_score, f1_score

In [35]:
predictions =[]
labels = []

model.eval()

with torch.no_grad():
    for image, label in validLoader:
        image = image.to(device)
        label = label.to(device)
        
        output = model(image)
        
        predictions.append(output.cpu().numpy())
        labels.append(label.cpu().numpy())
        
        


In [62]:
labelA = np.concatenate(labels)
predA = np.concatenate(predictions)

In [64]:
threshold = 0.5

In [73]:

for i in range(num_classes):
    
    label = labelA[:,i]
    pred = predA[:,i]
    
    auc = roc_auc_score(label, pred)
    
    ind = pred >= threshold
    
    pred[ind] = 1.0
    pred[~ind] = 0.0
    
    acc = accuracy_score(pred, label)
    
    
    
    print("{m: <26} : AUC - {a:<5} Acc - {b: <6} ".format(m=cols[(5+i)], a=np.round(auc,3), b=np.round(acc,3)))
    
    
    

No_Finding                 : AUC - 0.706 Acc - 0.917  
Enlarged_Cardiomediastinum : AUC - 0.502 Acc - 0.815  
Cardiomegaly               : AUC - 0.661 Acc - 0.836  
Lung_Opacity               : AUC - 0.655 Acc - 0.656  
Lung_Lesion                : AUC - 0.509 Acc - 0.891  
Edema                      : AUC - 0.773 Acc - 0.829  
Consolidation              : AUC - 0.503 Acc - 0.766  
Pneumonia                  : AUC - 0.522 Acc - 0.847  
Atelectasis                : AUC - 0.543 Acc - 0.686  
Pneumothorax               : AUC - 0.597 Acc - 0.881  
Pleural_Effusion           : AUC - 0.765 Acc - 0.777  
Pleural_Other              : AUC - 0.5   Acc - 0.909  
Fracture                   : AUC - 0.501 Acc - 0.896  
Support_Devices            : AUC - 0.793 Acc - 0.79   
